In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
from IPython.display import Markdown
import re
import datetime
from pykrx.stock import get_market_ticker_list, get_market_ticker_name
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.tools import tool
from langchain_core.documents.base import Document
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from pydantic import BaseModel, Field
from markitdown import MarkItDown
import dotenv
import os

dotenv.load_dotenv()

True

In [ ]:
import yfinance as yf

# 피엔티 , 137400

In [111]:
class State(TypedDict):
    pass

In [97]:
# 주식 DB를 만드는 함수 {"삼성전자":"005930, ... }

def create_stock_db():
    stock_dict = {}
    today = datetime.datetime.today()
    
    stock_list = get_market_ticker_list(today, market="KOSPI")

    for stock in stock_list:
        stock_dict.update({get_market_ticker_name(stock):stock})
    
    stock_list = get_market_ticker_list(today, market="KOSDAQ")

    for stock in stock_list:
        stock_dict.update({get_market_ticker_name(stock):stock})

    return stock_dict

In [98]:
stock_db = create_stock_db()

In [99]:
prompt = PromptTemplate.from_template("""
당신은 주식 이름 추출기입니다.
주어진 문장에서 주식이름만 추출하세요.

### 예시 1
query : 삼성전자의 최근 1년에 대해서 분석해주세요.

answer : 삼성전자

### 예시 2
query : AJ홀딩스우의 최근 실적은 얼마인가요?

answer : AJ홀딩스우

### 입력
query : {query}

answer : 

"""
)

llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0,)

In [ ]:
stock_name = "피엔티"

stock_code = stock_db[stock_name]


In [155]:
@tool
def search_stock(query):

    """
    주식 검색 도구입니다.
    결과값으로 데이터프레임이 반환됩니다.
    입력 쿼리에서 주식이름을 추출한 후 모든 주식 데이터를 가져옵니다.
    """


    prompt = PromptTemplate.from_template("""
        당신은 주식 이름 추출기입니다.
        주어진 문장에서 주식이름만 추출하세요.

        ### 예시 1
        query : 삼성전자의 최근 1년에 대해서 분석해주세요.

        answer : 삼성전자

        ### 예시 2
        query : AJ홀딩스우의 최근 실적은 얼마인가요?

        answer : AJ홀딩스우

        ### 입력
        query : {query}

        answer : 

        """
        )
    
    chain = prompt | llm | StrOutputParser()

    stock_name = chain.invoke({"query":query})

    try:
        stock_code = stock_db[stock_name.strip().upper()]   
    except:
        raise ValueError(f"종목명 : {stock_name}을/를 검색할 수 없습니다. 오탈자나 한국거래소에서 거래중인 주식인지 확인해주세요.")
    
    # 예: 삼성전자 (한국거래소는 뒤에 '.KS'를 붙임)
    ticker = yf.Ticker(stock_code+".KS")

    df = ticker.history(period="max") # 기간: '1d', '5d', '1mo', '1y', 'max' 등

    return df

In [152]:
df = search_stock("posco홀딩스의 최근 실적은 어떤가요?")

In [153]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-04 00:00:00+09:00,75453.618838,80802.942562,72638.185299,79958.312500,971800,0.0,0.0
2000-01-05 00:00:00+09:00,76016.696802,87841.516304,76016.696802,84181.453125,2885100,0.0,0.0
2000-01-06 00:00:00+09:00,84744.551579,85870.725022,79113.684364,81366.031250,659800,0.0,0.0
2000-01-07 00:00:00+09:00,81365.995824,83055.255253,80521.366110,81647.539062,511600,0.0,0.0
2000-01-10 00:00:00+09:00,82210.642592,82773.729185,79395.209626,80521.382812,412700,0.0,0.0


In [162]:
@tool
def web_search(query):
    """
    입력된 질의를 바탕으로 검색을 시도하는 도구입니다.
    기본적으로 5개의 검색 결과를 가져옵니다.
    """
    tool = TavilySearchResults()
    return tool

In [ ]:
tools = [PythonAstREPLTool(), TavilySearchResults(), search_stock]

In [166]:
llm_with_tools = llm.bind_tools(tools)

In [173]:
result = llm_with_tools.stream("피엔티의 최근 실적은 어떤가요?", stream_mode="values")

In [172]:
for step in result:
    print(step)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_eJlueOyRz6qlY1dDtoBjtKwa', 'function': {'arguments': '', 'name': 'search_stock'}, 'type': 'function'}]} response_metadata={} id='run-c19068de-2ea6-4e2c-addb-d100c1dbbffe' tool_calls=[{'name': 'search_stock', 'args': {}, 'id': 'call_eJlueOyRz6qlY1dDtoBjtKwa', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'search_stock', 'args': '', 'id': 'call_eJlueOyRz6qlY1dDtoBjtKwa', 'index': 0, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]} response_metadata={} id='run-c19068de-2ea6-4e2c-addb-d100c1dbbffe' tool_calls=[{'name': '', 'args': {}, 'id': None, 'type': 'tool_call'}] tool_call_chunks=[{'name': None, 'args': '{"', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'query', 'name': None}, 'type': None}]} response_

In [175]:
for step in result:
    step.pretty_print()

In [164]:
ToolNode(tools)

tools(tags=None, recurse=True, explode_args=False, func_accepts_config=True, func_accepts={'store': ('__pregel_store', None)}, tools_by_name={'python_repl_ast': PythonAstREPLTool(globals={}, locals={}), 'tavily_search_results_json': TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))), 'search_stock': StructuredTool(name='search_stock', description='주식 검색 도구입니다.\n결과값으로 데이터프레임이 반환됩니다.\n입력 쿼리에서 주식이름을 추출한 후 모든 주식 데이터를 가져옵니다.', args_schema=<class 'langchain_core.utils.pydantic.search_stock'>, func=<function search_stock at 0x0000028250B3E950>)}, tool_to_state_args={'python_repl_ast': {}, 'tavily_search_results_json': {}, 'search_stock': {}}, tool_to_store_arg={'python_repl_ast': None, 'tavily_search_results_json': None, 'search_stock': None}, handle_tool_errors=True, messages_key='messages')

In [157]:
web_search = TavilySearchResults()

In [161]:
web_search.invoke("피엔티는 어떤 기업인가요?")

[{'title': "[기업분석] 배터리랑 같이 크는 장비 업체 '피엔티' - 바이라인네트워크",
  'url': 'https://byline.network/2022/07/15-195/',
  'content': '피엔티는 이차전지를 구성하는 전극과 분리막, 전자소재 등을 제조하는 장비와 자동화 설비를 제공하는 국내 업체입니다. 2003년에 설립했고 2013년에',
  'score': 0.8282873},
 {'title': '회사소개 | (주)피엔티',
  'url': 'https://www.epnt.co.kr/company/',
  'content': '주식회사 피앤티는 철저한 장인정신과 미래를 개척하는 도전정신으로 감동을 주는 회사, 신뢰를 주는 롤투롤 설비 생산 기업 입니다.',
  'score': 0.7001659555555555},
 {'title': '보유종목점검 - 1. 피엔티 : 네이버 블로그',
  'url': 'https://blog.naver.com/chlbest/223714149861',
  'content': '피엔티, 현대차 의왕 연구소에 전고체 전극 장비 공급\u200b 피엔티 공장 조감도(사진=회사 홈페이지)이차전지 제조 장비 전문 기업인 피엔티(PNT)가 현대자동',
  'score': 0.57435945},
 {'title': '(주)피엔티 2025년 기업정보 | 직원수, 근무환경, 복리후생 등 - 사람인',
  'url': 'https://www.saramin.co.kr/zf_user/company-info/view/csn/Zm9qNG9WcGt6dlp0RWxvelMrSnVHUT09/company_nm/(%EC%A3%BC)%ED%94%BC%EC%97%94%ED%8B%B0',
  'content': '피엔티는 각종 롤투롤 컨버팅 기계를 주력으로 성장한 회사입니다. 웹가이딩, 장력제어, 오토터렛, 오토스플라이싱, 코팅, 칼렌더링, 슬리팅 등 기반이 되는 기술에 강점',
  'score': 0.48574239999999996},
 